## Теоретическая часть

**1.** Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации.  
- Чем они отличаются и чем они похожи?  
- Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
- В чем принципиальные отличия item-item рекомендаций от ALS?


**Ответ 1:**  
Отличаются:  
- Назначением
  - Если нужно провести рекламную кампанию, то к товарам хорошо бы подобрать аудиторию
  - Если товар был куплен, то хорошо бы сделать предложение для кросспродажи (чехол для iPad)
- Размерностью (могут отличаться на порядки, например магазин устройств Apple)

Похожи:
- Сравнивают одинаковые сущности
- Квадратные, а значит одни и те же алгоритмы мат. обработки должны подходить обоим матрицам

**Ответ 2:**  
Вероятно функции рекомендаций могут быть использованы обеими матрицами, если внутри нет какой-то специальной для этого типа объектов обработки.

**Ответ 3:**  
ALS - позволяет сэкономить место путем разбиения матрицы users-items на 2 матрицы
item-item - нет разбиения матриц

Предположение:
item-item - для explicit систем
ALS - для implicit систем



**2.** Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


1. **total_discount = retail_disc + coupon_disc + coupon_match_disc**
2. **total_discount_per_item = total_discount * sales_value**
3. **total_sum_discount = total_discount \* quantity \* sales_value**

**3.** Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


**Ответ**:  
1. Учитывает только факт покупки пользователем товара  
2. Требует больше времени/мощностей для вычислений  
3. Хуже интерпретируется чем item-item

**4.** Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

## Практическая часть


In [2]:
import implicit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import os, sys
from implicit.evaluation import precision_at_k
import warnings

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

warnings.filterwarnings('ignore')

In [3]:
def precision_at_k(recommended_list, bought_list, k=8):
    if type(bought_list) == str:
        bought_str_list = bought_list[1:-1].split(', ')
        bought_list = [int(item) for item in bought_str_list]

    if type(recommended_list) == str:
        bought_str_list = recommended_list[1:-1].split(', ')
        recommended_list = [int(item) for item in bought_str_list]

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [8]:
data = pd.read_csv('./data/retail_train.csv')

test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [10]:
result = pd.read_csv('./data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12695716, 8068934, 12428458, 434997, 858425]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13910611, 911878, 12301378, 9270394, 1109089]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.GridSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

In [17]:
items = pd.read_csv('./data/product.csv')
items.columns = [col.lower() for col in items.columns]
items.head(5)

,product_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [18]:
items.rename(columns={'product_id': 'item_id'}, inplace=True)
items.head(5)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
